In [1]:
import os
os.environ['USE_PYGEOS'] = '0'
import geopandas as gpd
from karstification import calc_karstification_for_HU12
from pandas import read_excel
from pynhd import pynhd
import py3dep
from shapely import box
import os
import string
import warnings

hr = pynhd.NHDPlusHR('huc12')
huc12 = pynhd.WaterData('wbd12', crs='epsg:4326')

def format_filename(s):
    """Take a string and return a valid filename constructed from the string.
Uses a whitelist approach: any characters not present in valid_chars are
removed. Also spaces are replaced with underscores.
 
Note: this method may produce invalid filenames such as ``, `.` or `..`
When I use this method I prepend a date string like '2009_01_15_19_46_32_'
and append a file extension like '.txt', so I avoid the potential of using
an invalid filename.
 
"""
    valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)
    filename = ''.join(c for c in s if c in valid_chars)
    filename = filename.replace(' ','_') # I don't like spaces in filenames.
    return filename


In [2]:
box_df = read_excel('bounding_boxes.xlsx')
bbox_zip = zip(box_df.x_min, box_df.y_min, box_df.x_max, box_df.y_max)

hucs_list = []
p_karst_list = []

for i, bbox in enumerate(bbox_zip):
    if i>1:
        print(asdf)
    boxname = box_df.Name[i]
    print("Processing", boxname)
    box_hucs12 = huc12.bybox(bbox)
    #Check available resolution
    avail = py3dep.check_3dep_availability(bbox)
    if avail['3m'] == True:
        dem_res = 3
        found_res = True
    elif avail['5m'] == True:
        dem_res = 5
        found_res = True
    elif avail['10m'] == True:
        dem_res = 10
        found_res = True
    else:
        found_res = False
    if found_res:
        print("Using dem resolution", str(dem_res))
        box_poly = box(*bbox)
        box_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[box_poly])
        box_dirname = format_filename(boxname)
        box_path = os.path.join('./qgis/', box_dirname)
        if not os.path.exists(box_path):
            os.makedirs(box_path)
        box_gdf.to_file(os.path.join(box_path, box_dirname + '.shp'))
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            box_hucs12.to_file(os.path.join(box_path, 'box_hucs.shp'))
        
        for hu_idx in box_hucs12.index:
            hu = box_hucs12.iloc[hu_idx]
            huc_num = hu.huc12
            print("HUC", huc_num)
            hu.crs = 'EPSG:4326'
            p_karst = calc_karstification_for_HU12(hu, boxname=box_dirname, dem_res=dem_res)
            hucs_list.append(huc_num)
            p_karst_list.append(p_karst)

            print('HU', huc_num, 'has', str(p_karst)[:5], 'percent internal karst drainage. Dem res was', dem_res)
        


Processing East of St. Louis #1, IL


Using dem resolution 3
HUC 071402040605
Failed to retrieve DEM.
./whitebox_tools --run="FillSingleCellPits" --dem='/home/mcoving/github/sinkhole_analysis/qgis/East_of_St._Louis_1_IL/071402040605/071402040605-USGS.tif' --output='/home/mcoving/github/sinkhole_analysis/qgis/East_of_St._Louis_1_IL/071402040605/071402040605-USGS-pitfill.tif' -v --compress_rasters=False

*********************************
* Welcome to FillSingleCellPits *
* Powered by WhiteboxTools      *
* www.whiteboxgeo.com           *
*********************************
Progress: 0%
Progress: 1%
Progress: 2%
Progress: 3%
Progress: 4%
Progress: 5%
Progress: 6%
Progress: 7%
Progress: 8%
Progress: 9%
Progress: 10%
Progress: 11%
Progress: 12%
Progress: 13%
Progress: 14%
Progress: 15%
Progress: 16%
Progress: 17%
Progress: 18%
Progress: 19%
Progress: 20%
Progress: 21%
Progress: 22%
Progress: 23%
Progress: 24%
Progress: 25%
Progress: 26%
Progress: 27%
Progress: 28%
Progress: 29%
Progress: 30%
Progress: 31%
Progress: 32%
Progress: 

In [56]:
box_hucs12


,geometry,objectid,tnmid,metasourceid,sourcedatadesc,sourceoriginator,sourcefeatureid,loaddate,gnis_id,areaacres,areasqkm,states,huc12,name,hutype,humod,tohuc,noncontributingareaacres,noncontributingareasqkm,globalid
0,"MULTIPOLYGON (((-90.60128 43.09439, -90.60027 ...",84153,{5E100092-E60F-48F1-B52B-0AECA567F1B5},None,None,None,None,2013-01-18T07:09:08Z,None,13961.69,56.50,WI,070700051403,Middle Fennimore Creek,S,TF,070700051405,0,0,{AB33AD02-E29C-11E2-8094-0021280458E6}
1,"MULTIPOLYGON (((-90.87636 43.06154, -90.87610 ...",84164,{0962EB20-3DC6-4B6E-A147-B6F4FE1A2B41},None,None,None,None,2013-01-18T07:09:08Z,None,13831.44,55.97,WI,070700051603,Big Green River,S,TF,070700051707,0,0,{AB34D7A2-E29C-11E2-8094-0021280458E6}
2,"MULTIPOLYGON (((-90.72836 43.13636, -90.72831 ...",84168,{9EC579FF-A8D1-4E72-97E5-7FA7EE8558E7},None,None,None,None,2013-01-18T07:09:08Z,None,11274.56,45.63,WI,070700051704,Sanders Creek,S,TF,070700051705,0,0,{AB354510-E29C-11E2-8094-0021280458E6}
3,"MULTIPOLYGON (((-90.73500 43.07697, -90.73482 ...",84170,{CBF12ACB-8B61-4C3D-8FD4-1778A225DC59},None,None,None,None,2013-01-18T07:09:08Z,None,10743.96,43.48,WI,070700051706,Crooked Creek,S,TF,070700051707,0,0,{AB357C4C-E29C-11E2-8094-0021280458E6}
4,"MULTIPOLYGON (((-90.75553 43.16394, -90.75538 ...",84171,{5B70D260-2ED9-4AEB-97E0-8C00261F6787},None,None,None,None,2013-01-18T07:09:08Z,None,24918.90,100.84,WI,070700051707,Clear Creek-Wisconsin River,S,TF,070700051802,0,0,{AB3593A8-E29C-11E2-8094-0021280458E6}


In [47]:
box_gdf.to_file('box1.shp')

,geometry
0,"POLYGON ((-90.63892 43.05695, -90.63892 43.122..."
